In [46]:
import datetime

import pandas as pd
import numpy as np

import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, Lasso

In [65]:
df = pd.read_csv('./output/v2.csv', index_col=0)
df.head()

,date,quantity,suppliers_currency,transformer_unit_price,dry_or_oil,rated_power_kVA,primary_winding,secondary_winding,no_load_loss,full_load_loss_75,full_load_loss_120,rated_volt_primary_side
0,2021-04-28,6.0,EUR,708000.0,Oil,19080.0,Copper,Copper,19002.544911,150154.155650,187698.233981,33000.0
3,2023-03-14,4.0,EUR,235000.0,Oil,11000.0,Copper,Copper,11825.516231,89149.546166,108715.888044,20000.0
8,2021-09-30,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.000000,138000.000000,146000.000000,21000.0
9,2021-09-30,5.0,EUR,267000.0,Oil,12000.0,Copper,Copper,8000.000000,138000.000000,146000.000000,24000.0
10,2020-12-02,1.0,EUR,56200.0,Dry,5000.0,Aluminum,Aluminum,5000.000000,53043.000000,61000.000000,11000.0


In [66]:
df = pd.get_dummies(
    df, columns=['dry_or_oil', 'primary_winding', 'secondary_winding']
).copy()

In [67]:
chosen_columns = [
    'quantity',
    'rated_power_kVA', 
    'no_load_loss', 
    # 'full_load_loss_75',
    # 'full_load_loss_120',
    'rated_volt_primary_side',
    'dry_or_oil_Dry',
    # 'dry_or_oil_Oil',
    'primary_winding_Aluminum',
    # 'primary_winding_Copper',
    'secondary_winding_Aluminum',
    # 'secondary_winding_Copper',
    # 'primary_material_price'
]

y = np.log(df['transformer_unit_price'])
# y = df['transformer_unit_price'] / df['transformer_unit_price'].mean()
X = df[chosen_columns].astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

en = ElasticNet(alpha=0.1, l1_ratio=0.5)
en.fit(X_scaled, y)

# Coefficients and Feature Importance
feature_importance = pd.DataFrame(
    {'Feature': X.columns, 
    'Lasso_Coefficient': en.coef_}
)

print(feature_importance.sort_values(by='Lasso_Coefficient'))

                      Feature  Lasso_Coefficient
4              dry_or_oil_Dry          -0.147524
6  secondary_winding_Aluminum          -0.029322
5    primary_winding_Aluminum          -0.000000
0                    quantity           0.044076
3     rated_volt_primary_side           0.161098
2                no_load_loss           0.170626
1             rated_power_kVA           0.408806


In [39]:
df['rated_volt_primary_side'].std()

np.float64(8446.190314844796)

In [40]:
df['rated_power_kVA'].std()

np.float64(5331.498402488682)

In [37]:
df['secondary_winding_Copper'].std()

np.float64(0.4170336944602445)